In [ ]:
#!/usr/bin/env python3
"""
Behavior Tree Visualization Tool
Menampilkan struktur dan status behavior tree secara real-time
"""

import rclpy
from rclpy.node import Node
from std_msgs.msg import String
import py_trees
from py_trees.display import unicode_tree
import os
import sys


class TreeVisualizer(Node):
    """Visualize behavior tree structure and status"""

    def __init__(self):
        super().__init__('tree_visualizer')

        # Subscribe to tree status topic
        self.create_subscription(
            String,
            '/behavior_tree/status',
            self.status_callback,
            10
        )

        self.get_logger().info('Tree Visualizer started')
        self.get_logger().info('Waiting for behavior tree status...')

    def status_callback(self, msg):
        """Callback untuk tree status updates"""
        # Clear screen (works on Unix-like systems)
        os.system('clear' if os.name == 'posix' else 'cls')

        print("=" * 60)
        print("BEHAVIOR TREE STATUS")
        print("=" * 60)
        print(msg.data)
        print("=" * 60)
        print("\nPress Ctrl+C to exit")


def create_tree_diagram():
    """Create a simple ASCII representation of the tree structure"""

    diagram = """
    Behavior Tree Structure:

    Root [Selector]
    ├── Battery Management [Sequence]
    │   ├── Is Battery Low? [Condition]
    │   └── Go to Charging Station [Action]
    │
    ├── Obstacle Avoidance [Sequence]
    │   ├── Obstacle Detected? [Condition]
    │   └── Avoid Obstacle [Action]
    │
    ├── Target Approach [Sequence]
    │   ├── Target Detected? [Condition]
    │   └── Approach Target [Action]
    │
    └── Patrol Waypoints [Action]

    Legend:
    ✓ SUCCESS   - Node succeeded
    ✗ FAILURE   - Node failed
    ⟳ RUNNING   - Node is executing
    ○ INVALID   - Node not yet executed
    """

    return diagram


def print_tree_info():
    """Print information about the behavior tree"""

    print("\n" + "=" * 60)
    print("BEHAVIOR TREE INFORMATION")
    print("=" * 60)

    print(create_tree_diagram())

    print("\nNode Descriptions:")
    print("-" * 60)

    descriptions = {
        "Battery Management": "Monitors battery level and initiates charging when low",
        "Is Battery Low?": "Checks if battery is below threshold (default 20%)",
        "Go to Charging Station": "Navigates robot to charging station at origin",
        "Obstacle Avoidance": "Detects and avoids obstacles in robot's path",
        "Obstacle Detected?": "Checks if obstacle is within safe distance",
        "Avoid Obstacle": "Executes avoidance maneuver (back up and turn)",
        "Target Approach": "Approaches detected target object",
        "Target Detected?": "Checks if target object is visible",
        "Approach Target": "Moves toward detected target",
        "Patrol Waypoints": "Default behavior - patrols predefined waypoints"
    }

    for name, desc in descriptions.items():
        print(f"\n• {name}")
        print(f"  {desc}")

    print("\n" + "=" * 60)
    print("\nBehavior Priorities (top to bottom):")
    print("1. Battery Management (highest)")
    print("2. Obstacle Avoidance")
    print("3. Target Approach")
    print("4. Patrol (default/lowest)")
    print("=" * 60)


def main(args=None):
    # Print static tree structure first
    print_tree_info()

    print("\nStarting real-time visualization...")
    print("Waiting for behavior tree node to publish status...\n")

    rclpy.init(args=args)
    node = TreeVisualizer()

    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        print("\n\nShutting down visualizer...")
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()